In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import simplejson as json
import geocoder
import requests
import csv
import re

In [2]:
water = pd.read_csv('River-2019.csv')
water = water.rename(columns={"採樣分區": "sampling_zone",
                              "河川": "river",
                              "縣市": "city",
                              "測站名稱": "station",
                              "水體分類等級": "water_body_type",
                              "採樣日期": "sampling_date",
                              "測站編號": "station_no",
                              "河川污染指數": "river_pollution_index",
                              "氣溫": "air_temp",
                              "水溫": "water_temp",
                              "酸鹼值": "PH",
                              "導電度": "conductivity",
                              "溶氧(滴定法)": "dissolved_oxygen_titration",
                              "溶氧(電極法)": "dissolved_oxygen_electrode_method",
                              "溶氧飽和度": "dissolved_oxygen_saturation",
                              "生化需氧量": "biochemical_oxygen_demand",
                              "化學需氧量": "cod",
                              "懸浮固體": "suspended_solids",
                              "大腸桿菌群": "EColi",
                              "氨氮": "ammonia_nitrogen",
                              "氯鹽": "chloride_salt",
                              "總磷": "total_phosphorus",
                              "總有機碳": "total_organic_carbon",
                              "硝酸鹽氮": "nitrate_nitrogen",
                              "亞硝酸鹽氮": "nitrite nitrogen",
                              "鎘": "cadmium",
                              "鉛": "lead",
                              "六價鉻": "hexavalent_chromium",
                              "砷": "arsenic",
                              "汞": "HG",
                              "銅": "copper",
                              "鋅": "zinc",
                              "錳": "manganese",
                              "銀": "silver",
                              "鎳": "nickel",
                              "硒": "selenium",
                              "濁度": "turbidity",
                              "鉻": "chromium",
                              "總氮": "total_nitrogen",
                              "總凱氏氮": "total_kjeldhal_nitrogen",
                              "備註": "remarks"
                              })
water["sampling_date"] = water["sampling_date"].str.replace('上午.', 'AM ', regex=True)
water["sampling_date"] = water["sampling_date"].str.replace('下午.', 'PM ', regex=True)
water['sampling_date'] = pd.to_datetime(water['sampling_date'], format='%Y/%m/%d %p %I:%M:%S')
water['sampling_month_year'] = water['sampling_date'].dt.strftime('%B %Y')
water = water.iloc[1:] #remove first row with data type labels
water = water.astype({'station_no': 'int32'})
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows",10)
water

,sampling_zone,river,city,station,water_body_type,sampling_date,station_no,river_pollution_index,air_temp,water_temp,PH,conductivity,dissolved_oxygen_titration,dissolved_oxygen_electrode_method,dissolved_oxygen_saturation,biochemical_oxygen_demand,cod,suspended_solids,EColi,ammonia_nitrogen,chloride_salt,total_phosphorus,total_organic_carbon,nitrate_nitrogen,nitrite nitrogen,cadmium,lead,hexavalent_chromium,arsenic,HG,copper,zinc,manganese,silver,nickel,selenium,turbidity,chromium,total_nitrogen,total_kjeldhal_nitrogen,remarks,sampling_month_year
1,雙溪流域,雙溪,新北市,新寮大橋,甲,2019-12-03 07:55:00,1034,1.0,15.1,16.7,7.03,83,--,10.0,102.7,<1.0,<4.0,9.1,30000,0.02,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,December 2019
2,雙溪流域,雙溪,新北市,新寮大橋,甲,2019-11-07 07:55:00,1034,1.0,21.2,20.6,7.12,87,--,9.1,102.1,<1.0,<4.0,1.0,910000,0.01,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,November 2019
3,雙溪流域,雙溪,新北市,新寮大橋,甲,2019-10-03 07:55:00,1034,1.0,26.3,23.7,7.04,76,--,8.7,103.0,<1.0,<4.0,2.8,24000,0.09,--,0.061,--,0.40,--,<0.001,<0.003,<0.002,<0.0003,<0.0003,<0.001,0.010,0.008,<0.001,<0.005,--,--,--,--,--,NaN,October 2019
4,雙溪流域,雙溪,新北市,新寮大橋,甲,2019-09-03 07:55:00,1034,1.0,29.2,26.6,7.42,95,--,8.3,103.8,<1.0,5.2,3.0,41000,0.13,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,September 2019
5,雙溪流域,雙溪,新北市,新寮大橋,甲,2019-08-02 07:55:00,1034,1.0,33.4,28.5,7.60,111,--,7.5,97.4,1.1,9.8,9.2,2400,0.11,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,August 2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3632,磺溪流域,磺溪,新北市,四十號橋,乙,2019-05-06 13:25:00,1033,1.0,22.1,21.0,4.32,264,--,8.7,97.4,<1.0,8.2,9.1,590,<0.01,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,May 2019
3633,磺溪流域,磺溪,新北市,四十號橋,乙,2019-04-08 14:15:00,1033,1.0,27.9,27.0,4.36,271,--,7.8,98.3,<1.0,5.3,4.5,190,0.04,--,0.042,--,0.44,--,<0.001,<0.003,<0.002,0.0011,<0.0003,0.003,0.021,0.122,<0.001,<0.005,--,--,--,--,--,NaN,April 2019
3634,磺溪流域,磺溪,新北市,四十號橋,乙,2019-03-06 12:15:00,1033,1.0,20.1,19.5,4.46,245,--,8.7,95.1,<1.0,4.3,6.1,750,0.02,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,March 2019
3635,磺溪流域,磺溪,新北市,四十號橋,乙,2019-02-11 15:35:00,1033,2.3,16.8,17.2,4.40,228,--,9.4,96.1,8.9,26.2,15.4,1600,0.12,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,NaN,February 2019


In [3]:
'''
Create a new dataframe containing only unique location information 
'''
locations = water[['sampling_zone', 'river', 'city', 'station', 'station_no']] #remove 'water_body_type' 
locations = locations.drop_duplicates()
locations.dropna(inplace=True)
locations['city_river'] = locations['city'] + water['river']
#locations['city_river_to_review'] = locations['city_river'].str.contains(re.escape('('), na=False)
locations['lat'] = ''     #fill in station Gis_Y
locations['lon'] = ''     #fill in station Gis_X
locations['address'] = '' #fill in station address

'''
Fill in station location with data from https://wq.epa.gov.tw/WS_WQEPA/ws_wqepa.asmx/GetWQEPASampleData_River (waterbody_id: 1)
'''
tree = ET.parse('river_stations.xml')

for fact in tree.iter(tag = 'dt_tmp'):
    locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'lat'] = fact.find('Gis_Y').text
    locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'lon'] = fact.find('Gis_X').text
    if fact.find('Station_Address') != None:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'address'] = fact.find('Station_Address').text
    else:
        locations.loc[locations['station_no'] == int(fact.find('Station_ID1').text), 'address'] = fact.find('Station_Name').text
        
with open('river_stations.json', 'w', encoding='utf-8') as file:
    locations.to_json(file, orient='records', indent=4, force_ascii=False)
locations

,sampling_zone,river,city,station,station_no,city_river,lat,lon,address
1,雙溪流域,雙溪,新北市,新寮大橋,1034,新北市雙溪,25.0317969,121.8347690,新北市雙溪區上林村北38號道
13,雙溪流域,雙溪,新北市,貢寮大橋,1035,新北市雙溪,25.0207100,121.9095289,新北市貢寮區102丙公路
25,得子口溪流域,得子口溪,宜蘭縣,竹安橋,1679,宜蘭縣得子口溪,24.8413333,121.8233611,宜蘭縣頭城鎮台2線北部濱海公路
37,得子口溪流域,得子口溪,宜蘭縣,美慶山莊,1332,宜蘭縣得子口溪,24.8314511,121.7470589,宜蘭縣礁溪鄉五峰路
49,得子口溪流域,得子口溪,宜蘭縣,七結橋(取代得子口橋),1221,宜蘭縣得子口溪,24.8031789,121.7651450,宜蘭縣礁溪鄉宜6鄉道(七結路)
...,...,...,...,...,...,...,...,...,...
3577,淡水河流域,淡水河本流,新北市,關渡大橋,1003,新北市淡水河本流,25.1248169,121.4572369,新北市八里區成泰路四段與縣103號道交會處附近
3589,淡水河流域,淡水河本流,新北市,淡水河口,1004,新北市淡水河本流,25.1709011,121.4227450,新北市淡水區
3601,磺溪流域,磺溪,新北市,上磺溪橋,1031,新北市磺溪,25.1792090,121.5822730,新北市金山區三和村2號公路大油坑旁
3613,磺溪流域,磺溪,新北市,磺溪橋,1032,新北市磺溪,25.2269530,121.6306240,新北市金山區台2線(中山路)


In [4]:
'''
Cleanup city_river river names in locations dataframe
We are assuming that stations will fall along river borders, we can use station locations to confirm that river locations are mapped 
'''

locations = locations.append(
    [{'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'老街溪, Dayuan District, Taoyuan, 33756, Taiwan','lat':'None',
                              'lon':'None', 'address': 'None'
    },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'老街溪, Longtan District, Taoyuan, 325, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
    },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'Laojie River, Longtan District, Taoyuan, 325, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
                             },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'茄苳溪, Bade District, Taoyuan, 33058, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
                             },
     
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'茄苳溪, Bade District, Taoyuan, 33453, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
                             },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'大坑崁溪, Pingzhen District, Taoyuan, 32559, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'石門大圳, Pingzhen District, Taoyuan, 33551, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'霄裡溪, Xinpu, Hsinchu County, Taiwan Province, 305, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'上坪溪, Zhudong, Hsinchu County, Taiwan Province, 311, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'南港溪, Zaoqiao, Miaoli County, Taiwan Province, 361, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'老庄溪, Miaoli County, Taiwan Province, 369, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旱溪, Hanxi Village, East District, Taichung, 401, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旱溪排水, Dali District, Taichung, 41473, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'南投貓羅溪', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旱溪排水, Dali District, Taichung, 41283, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旱溪, Dunnan Village, Houli District, Taichung, 42147, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旱溪, Xintian Village, Tanzi District, Taichung, 40647, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旱溪, Wuri District, Taichung, 414, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'谷溪', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'旗山溪, Jiaxian District, Kaohsiung, 847, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'東港溪, Neipu, Pingtung County, Taiwan Province, 912, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'牛稠溪, Pingtung County, Taiwan Province, 90049, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'Taiping River, Tai''an Village, Beinan, Taitung County, Taiwan Province, 954, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'Luye River, Taoyuan Village, Yanping, Taitung County, Taiwan Province, 953, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'花蓮太平溪', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'Lele River, Zhuoxi, Hualien County, Taiwan Province, 982, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'荖溪, Shoufeng, Hualien County, Taiwan Province, 974, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'壽豐溪(知亞干溪), Wanrong, Hualien County, Taiwan Province, 979, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'七家灣溪, Heping District, Taichung, 42413, Taiwan', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'南澳南溪', 'lat':'None',
                              'lon':'None', 'address': 'None'
     },
     {'sampling_zone':'None', 'river':'None', 'city': 'None', 'station':'None',
                              'station_no':'None', 'city_river':'林尾溪', 'lat':'None',
                              'lon':'None', 'address': 'None'
     }
    ], ignore_index=True)

#TAIPEI
locations.loc[locations['city_river'] == '臺北市淡水河本流', 'city_river'] = '新北市淡水河'

#NEW TAIPEI
locations.loc[locations['city_river'] == '新北市淡水河本流', 'city_river'] = '新北市淡水河'
locations.loc[locations['city_river'] == '新北市新店溪', 'city_river'] = '臺北市新店溪'
locations.loc[locations['city_river'] == '新北市景美溪', 'city_river'] = '臺北市景美溪'
locations.loc[locations['city_river'] == '新北市基隆河', 'city_river'] = '基隆市基隆河'

#TAOYUAN
locations.loc[locations['city_river'] == '桃園市福興溪', 'city_river'] = '新竹縣福興溪'
locations.loc[locations['city_river'] == '桃園市老街溪', 'city_river'] = '中壢市老街溪'
locations.loc[locations['city_river'] == '桃園市茄苳溪(桃)', 'city_river'] = '茄苳溪, Taoyuan, Taoyuan District, Taoyuan, 330, Taiwan'
locations.loc[locations['city_river'] == '桃園市南崁溪', 'city_river'] = 'Nankan River, Guishan District, Taoyuan, 333, Taiwan'


#HSINCHU
locations.loc[locations['city_river'] == '新竹市客雅溪', 'city_river'] = 'keya creek hsinchu'
locations.loc[locations['city_river'] == '新竹縣鹽港溪', 'city_river'] = '新竹市鹽港溪'
locations.loc[locations['city_river'] == '新竹縣客雅溪', 'city_river'] = 'keya creek hsinchu'

#MIAOLI
locations.loc[locations['city_river'] == '苗栗縣南港溪(苗)', 'city_river'] = '苗栗縣中港溪'

#TAICHUNG
locations.loc[locations['city_river'] == '臺中市烏溪', 'city_river'] = '旱溪排水大里 41473'
locations.loc[locations['city_river'] == '臺中市大安溪', 'city_river'] = '苗栗縣大安溪'
locations.loc[locations['city_river'] == '彰化縣烏溪', 'city_river'] = '臺中市大里溪'

#NANTOU
locations.loc[locations['city_river'] == '南投縣清水溪(投)', 'city_river'] = '清水溪, Nantou County, Taiwan Province, 55759, Taiwan'
locations.loc[locations['city_river'] == '南投縣北港溪(投)', 'city_river'] = '南港溪, Nantou County, Taiwan Province, 544, Taiwan'

#YUNLIN
locations.loc[locations['city_river'] == '雲林縣北港溪(雲)', 'city_river'] = '雲林縣北港溪'

#HUALIEN
locations.loc[locations['city_river'] == '花蓮縣吉安溪', 'city_river'] = '吉安溪 (七腳川溪), Hualien County, Taiwan Province, 97342, Taiwan'


locations

,sampling_zone,river,city,station,station_no,city_river,lat,lon,address
0,雙溪流域,雙溪,新北市,新寮大橋,1034,新北市雙溪,25.0317969,121.8347690,新北市雙溪區上林村北38號道
1,雙溪流域,雙溪,新北市,貢寮大橋,1035,新北市雙溪,25.0207100,121.9095289,新北市貢寮區102丙公路
2,得子口溪流域,得子口溪,宜蘭縣,竹安橋,1679,宜蘭縣得子口溪,24.8413333,121.8233611,宜蘭縣頭城鎮台2線北部濱海公路
3,得子口溪流域,得子口溪,宜蘭縣,美慶山莊,1332,宜蘭縣得子口溪,24.8314511,121.7470589,宜蘭縣礁溪鄉五峰路
4,得子口溪流域,得子口溪,宜蘭縣,七結橋(取代得子口橋),1221,宜蘭縣得子口溪,24.8031789,121.7651450,宜蘭縣礁溪鄉宜6鄉道(七結路)
...,...,...,...,...,...,...,...,...,...
331,None,None,None,None,None,"荖溪, Shoufeng, Hualien County, Taiwan Province,...",None,None,None
332,None,None,None,None,None,"壽豐溪(知亞干溪), Wanrong, Hualien County, Taiwan Pro...",None,None,None
333,None,None,None,None,None,"七家灣溪, Heping District, Taichung, 42413, Taiwan",None,None,None
334,None,None,None,None,None,南澳南溪,None,None,None


In [7]:

'''
Using nominatim package to get river locations

https://nominatim.org/release-docs/develop/api/Search/#examples
https://nominatim.openstreetmap.org/search.php?q=%E6%96%B0%E7%AB%B9%E7%B8%A3%E9%B3%B3%E5%B1%B1%E6%BA%AA&polygon_geojson=1&viewbox=
https://nominatim.openstreetmap.org/search?q=%E6%96%B0%E7%AB%B9%E7%B8%A3%E9%B3%B3%E5%B1%B1%E6%BA%AA&format=json&polygon_geojson=1&addressdetails=1&limit=1
'''
geocoded_locations_nominatum = []
unique_locations_nominatum = []

    
# Create CSV file with unique city-river list
with open('rivers_geocoded_nominatum.csv', 'w', newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['location', 'name', 'type', 'lat', 'lon'])
    for location in locations['city_river'].unique():
        result = ''
        geo_json = {}
        resp = requests.get('https://nominatim.openstreetmap.org/search?q='+ location +'&format=json&polygon_geojson=1&addressdetails=1')
        if resp.status_code == 200:
            if resp.json():
                if location == '南港溪, Nantou County, Taiwan Province, 544, Taiwan':
                    result = resp.json()[1]
                elif location == '清水溪, Nantou County, Taiwan Province, 55759, Taiwan':
                    result = resp.json()[1]
                elif location == '谷溪':
                    result = resp.json()[2]                    
                else:
                    result = resp.json()[0]
                geo_json["type"] = "Feature"
                geo_json["geometry"] = result['geojson']
                geo_json["properties"] = {"location": location, "display": result['display_name'] }
                print(location, result['display_name'], result['type'], result['lat'], result['lon'])
                geocoded_locations_nominatum.append(geo_json)           
                spamwriter.writerow([location, result['display_name'], result['type'], result['lat'], result['lon']])
            else: 
                print(location + " not found")
                spamwriter.writerow([location, 'not found'])
        else:
            spamwriter.writerow([location, 'not found'])

# Create JSON file with geocoded data for each city-river  
geocoded_locations = {'data': geocoded_locations_nominatum}
with open('rivers_geocoded_nominatum.json', 'w', encoding='utf-8') as f:
    json.dump(geocoded_locations, f, ensure_ascii=False, sort_keys=True, indent=4)

新北市雙溪 雙溪, 雙溪區, 新北市, 22742, Taiwan river 25.0331832 121.85431
宜蘭縣得子口溪 得子口溪, 宜蘭縣, 臺灣省, 26244, Taiwan river 24.8154006 121.7671014
宜蘭縣蘭陽溪 蘭陽溪, 大同鄉, 宜蘭縣, 臺灣省, 267, Taiwan river 24.6098966 121.5327044
宜蘭縣冬山河 冬山河, 冬山鄉, 宜蘭縣, 臺灣省, 269, Taiwan river 24.6445902 121.7914189
宜蘭縣新城溪 新城溪, 宜蘭縣, 臺灣省, 26948, Taiwan river 24.594269 121.7927345
宜蘭縣蘇澳溪 蘇澳溪, 宜蘭縣, 臺灣省, 270, Taiwan river 24.573872 121.841303
宜蘭縣南澳溪 南奧溪, 宜蘭縣, 臺灣省, 27244, Taiwan river 24.5329943 121.7169998
宜蘭縣和平溪 和平溪, 宜蘭縣, 臺灣省, Taiwan river 24.3981148 121.5843567
花蓮縣立霧溪 立霧溪, 花蓮縣, 臺灣省, 972, Taiwan river 24.1782003 121.4765221
花蓮縣三棧溪 三棧溪, 花蓮縣, 臺灣省, 971, Taiwan river 24.0831072 121.5541238
花蓮縣美崙溪 美崙溪, 花蓮縣, 臺灣省, 97059, Taiwan river 24.0008125 121.5651216
吉安溪 (七腳川溪), Hualien County, Taiwan Province, 97342, Taiwan 吉安溪 (七腳川溪), 花蓮縣, 臺灣省, 97342, Taiwan stream 23.9828851 121.5703291
花蓮縣木瓜溪 木瓜溪, 秀林鄉, 花蓮縣, 臺灣省, 972, Taiwan river 23.9703325 121.4801381
花蓮縣花蓮溪 花蓮溪, 花蓮縣, 臺灣省, 979, Taiwan river 23.7461256 121.37623
花蓮縣秀姑巒溪 秀姑巒溪, 玉里鎮, 花蓮縣, 臺灣省, 981, Taiwan ri

七家灣溪, Heping District, Taichung, 42413, Taiwan 七家灣溪, 和平區, 臺中市, 42413, Taiwan stream 24.4067802 121.3142811
南澳南溪 南澳南溪, 南澳鄉, 宜蘭縣, 臺灣省, 272, Taiwan river 24.4247765 121.7355698
林尾溪 林尾溪, 礁溪鄉, 宜蘭縣, 臺灣省, 26244, Taiwan river 24.814497 121.7445166


In [ ]:

'''
river_pollution_index stats

https://medium.com/dunder-data/finding-the-percentage-of-missing-values-in-a-pandas-dataframe-a04fa00f84ab
'''
# Replace -- with np.nan
water['river_pollution_index'] = water['river_pollution_index'].replace('--', np.nan)



def stats(x):
    return x.isna().sum() #x.isna().mean().round(4) * 100

stats.__name__ = 'RPI_number_of_nans'

water.groupby([water['sampling_month_year']]).agg({
     'air_temp': ['min', 'max'],
     'river_pollution_index': stats})



#rpi_missing = water['river_pollution_index'].isna()
#water = water['river_pollution_index'].dropna()
#water.sort_values(by='sampling_date',inplace=True) #ascending=False

#tests = water[(water['river_pollution_index'].isna())]
#rpi_stats = water.groupby([water['sampling_month_year']]).agg({'river_pollution_index': [np.max, np.min]})
#rpi_stats.to_json('rpi_stats.json', indent=4)

#tests

#rpi_stats = water.loc[water['sampling_date'].dt.strftime('%B %Y') == 'August 2019']
#rpi_stats.loc[(rpi_stats['river_pollution_index'].isna())]

#water.groupby([water['river_pollution_index']]).size()
#water['river_pollution_index'].unique()
#water.groupby([water['sampling_date'].dt.strftime('%B %Y'), water['city'], water['station'] ]).agg({'river_pollution_index': [np.max, np.min]})
#water['location_to_geocode'] = water['location_to_geocode'].unique()




In [ ]:
'''
- Use site code to link river 2019 data and API results so we can get the station lat and lon and plot on top of the rivers
- Map lat and lang back to river 2019 data so can be drawn on map

https://stackoverflow.com/questions/18175489/sending-soap-request-using-python-requests
https://wq.epa.gov.tw/WS_WQEPA/ws_wqepa.asmx/GetWQEPASampleData_River


url="https://wq.epa.gov.tw/WS_WQEPA/ws_wqepa.asmx?WSDL"
#resp = requests.get(url)

headers = {'content-type': 'text/xml; charset=utf-8', 
           'Host': 'wq.epa.gov.tw', 
           'SOAPAction': ''"http://localhost/WS_WQEpa/ws_wqepa/GetWaterBody_Station"''}


body = """<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <GetWaterBody_Station xmlns="http://localhost/WS_WQEpa/ws_wqepa">
      <waterbody_id>string</waterbody_id>
    </GetWaterBody_Station>
  </soap:Body>
</soap:Envelope>"""

response = requests.post(url,data=body,headers=headers)
print(response.content)
'''   

In [31]:
resp = requests.get('https://nominatim.openstreetmap.org/search?q=Taiping River, Tai''an Village, Beinan, Taitung County, Taiwan Province, 954, Taiwan&format=json&polygon_geojson=1&addressdetails=1')
resp.json()[0]

{'place_id': 273302736,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 762580977,
 'boundingbox': ['22.796856', '22.8191302', '121.0451577', '121.0951091'],
 'lat': '22.8066536',
 'lon': '121.0695349',
 'display_name': '太平溪, 泰安村, 卑南鄉, 臺東縣, 臺灣省, 954, Taiwan',
 'class': 'waterway',
 'type': 'river',
 'importance': 0.6050000000000001,
 'address': {'waterway': '太平溪',
  'city_district': '泰安村',
  'town': '卑南鄉',
  'county': '臺東縣',
  'state': '臺灣省',
  'postcode': '954',
  'country': 'Taiwan',
  'country_code': 'tw'},
 'geojson': {'type': 'LineString',
  'coordinates': [[121.0451577, 22.8191302],
   [121.0452908, 22.8182791],
   [121.0457713, 22.8174575],
   [121.0459797, 22.8169373],
   [121.0468248, 22.8162063],
   [121.0469753, 22.8158381],
   [121.0472879, 22.8152085],
   [121.0477423, 22.8149151],
   [121.0483501, 22.8143895],
   [121.049172, 22.814008],
   [121.0501445, 22.8139333],
   [121.0518174, 22.814024],
   [121.